# Autograd-自动求导机制

autograd包为张量上的所有操作提供了自动求导。 它是一个在运行时定义的框架，这意味着反向传播是根据你的代码来确定如何运行，并且每次迭代可以是不同的。

### 张量(Tensor) 

In [1]:
import torch

创建一个张量x并设置 requires_grad=True 用来追踪它的计算历史

In [2]:
x=torch.ones(2,2,requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


对张量x进行操作并查看结果y的grad_fn属性(被自动生成) 

In [3]:
y=x+2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


对张量y进行操作

In [4]:
z=y*y*3
out=z.mean()
print(z,"\n",out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) 
 tensor(27., grad_fn=<MeanBackward0>)


.requires_grad_( ... ) 可以改变现有张量的 requires_grad属性。 如果没有指定的话，默认输入的flag是 False。

In [6]:
a=torch.randn(2,2)
a=((a*3)/(a-1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b=(a*a).sum()
print(b,b.grad_fn)

False
True
tensor(3241.3933, grad_fn=<SumBackward0>) <SumBackward0 object at 0x00000178016B8828>


### 梯度

反向传播 out是一个纯量（scalar），out.backward() 等于out.backward(torch.tensor(1))。

In [7]:
out.backward()

In [8]:
print(x.grad) # 上面的张量x经过一系列操作得到结果out，然后执行out.backward()进行反向传播

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


使用autograd执行更多的操作

In [33]:
x=torch.randn(3,requires_grad=True)
y=x*2
print("init:",y)

while y.data.norm()<1000:
    y=y*2

print(y)    

init: tensor([ 1.5748, -0.1858,  2.8354], grad_fn=<MulBackward0>)
tensor([ 806.2749,  -95.1377, 1451.7339], grad_fn=<MulBackward0>)


In [35]:
print("init:",x)
print(x.grad)
gradients=torch.tensor([0.1,1.0,0.0001],dtype=torch.float)
y.backward(gradients)
print(x)
print(x.grad)

init: tensor([ 0.7874, -0.0929,  1.4177], requires_grad=True)
None
tensor([ 0.7874, -0.0929,  1.4177], requires_grad=True)
tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


如果.requires_grad=True但是你又不希望进行autograd的计算， 那么可以将变量包裹在 with torch.no_grad()中:

In [38]:
print(x.requires_grad)
print((x **2 ).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


# 更新自动求导内容 ^_^"

深度学习的算法本质上是通过反向传播求导数，而PyTorch的autograd模块则实现了此功能。在Tensor上的所有操作，autograd都能为它们自动提供微分，避免了手动计算导数的复杂过程。

**从0.4起, Variable 正式合并入Tensor, Variable 本来实现的自动微分功能，Tensor就能支持。读者还是可以使用Variable(tensor), 但是这个操作其实什么都没做。**

所以，以后的代码建议直接使用Tensor，因为官方文档中已经将Variable设置成过期模块

要想使得Tensor使用autograd功能，只需要设置tensor.requries_grad=True

In [1]:
import torch

在张量创建时，通过设置 requires_grad 标识为Ture来告诉Pytorch需要对该张量进行自动求导，PyTorch会记录该张量的每一步操作历史并自动计算

In [2]:
x =  torch.rand(5,5,requires_grad=True)
print(x)

tensor([[0.9089, 0.2767, 0.5271, 0.2888, 0.2459],
        [0.4739, 0.5858, 0.1260, 0.1775, 0.2105],
        [0.8712, 0.5196, 0.3228, 0.7240, 0.9289],
        [0.4810, 0.0644, 0.7885, 0.1979, 0.0199],
        [0.3308, 0.4363, 0.7468, 0.6913, 0.0291]], requires_grad=True)


In [4]:
y = torch.rand(5,5,requires_grad=True)
y

tensor([[0.5102, 0.4021, 0.1438, 0.5454, 0.3878],
        [0.7224, 0.6090, 0.2609, 0.4487, 0.7966],
        [0.5154, 0.8883, 0.5591, 0.6081, 0.7934],
        [0.0936, 0.2627, 0.9258, 0.8730, 0.6391],
        [0.3332, 0.2754, 0.4257, 0.3044, 0.6081]], requires_grad=True)

可以看到，该张量的grad_fn已经被赋予了一个新的函数 ..@_@|||.. 下面将调用反向传播函数计算其梯度

In [6]:
z = torch.sum(x+y)
z

tensor(23.9060, grad_fn=<SumBackward0>)

### 简单的自动求导

In [7]:
z.backward()
print(x.grad,'\n',y.grad) # 查看x和y的梯度

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]) 
 tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])


### 复杂的自动求导

In [17]:
x = torch.rand(5, 5, requires_grad=True)
y = torch.rand(5, 5, requires_grad=True)
z= x**2+y**3
print(x,'\n',y,'\n',z)

tensor([[0.5868, 0.3230, 0.7076, 0.2020, 0.3497],
        [0.5547, 0.8838, 0.2279, 0.0338, 0.4128],
        [0.6966, 0.9762, 0.7723, 0.0204, 0.4880],
        [0.9451, 0.5801, 0.9444, 0.8114, 0.6059],
        [0.3542, 0.1401, 0.7452, 0.0872, 0.2609]], requires_grad=True) 
 tensor([[0.8738, 0.3660, 0.1345, 0.1510, 0.4227],
        [0.4882, 0.4784, 0.2796, 0.6904, 0.4370],
        [0.6893, 0.0617, 0.0803, 0.0773, 0.9006],
        [0.4791, 0.5440, 0.0504, 0.6436, 0.0191],
        [0.5383, 0.0211, 0.6915, 0.7350, 0.1388]], requires_grad=True) 
 tensor([[1.0114e+00, 1.5331e-01, 5.0315e-01, 4.4231e-02, 1.9782e-01],
        [4.2402e-01, 8.9056e-01, 7.3789e-02, 3.3029e-01, 2.5388e-01],
        [8.1273e-01, 9.5319e-01, 5.9691e-01, 8.7727e-04, 9.6861e-01],
        [1.0032e+00, 4.9751e-01, 8.9202e-01, 9.2500e-01, 3.6712e-01],
        [2.8146e-01, 1.9639e-02, 8.8606e-01, 4.0463e-01, 7.0765e-02]],
       grad_fn=<AddBackward0>)


我们的返回值不是一个scalar，所以需要输入一个大小相同的张量作为参数，这里我们用ones_like()函数根据x生成一个张量

In [18]:
z.backward(torch.ones_like(x))
print(x.grad,'\n',y.grad)
# x梯度 = 2*x
# y梯度 = 3*y*y

tensor([[1.1736, 0.6459, 1.4152, 0.4039, 0.6995],
        [1.1094, 1.7676, 0.4557, 0.0676, 0.8257],
        [1.3932, 1.9524, 1.5445, 0.0408, 0.9760],
        [1.8902, 1.1603, 1.8888, 1.6228, 1.2118],
        [0.7084, 0.2802, 1.4905, 0.1745, 0.5219]]) 
 tensor([[2.2905e+00, 4.0178e-01, 5.4270e-02, 6.8413e-02, 5.3590e-01],
        [7.1491e-01, 6.8658e-01, 2.3457e-01, 1.4301e+00, 5.7283e-01],
        [1.4253e+00, 1.1410e-02, 1.9362e-02, 1.7923e-02, 2.4333e+00],
        [6.8875e-01, 8.8770e-01, 7.6086e-03, 1.2427e+00, 1.0939e-03],
        [8.6941e-01, 1.3388e-03, 1.4346e+00, 1.6205e+00, 5.7802e-02]])


我们可以使用with torch.no_grad()上下文管理器临时禁止对已设置requires_grad=True的张量进行自动求导。这个方法在测试集计算准确率的时候会经常用到，例如

In [19]:
with torch.no_grad():
    print((x+y*2).requires_grad)

False
